# Write ApRES xarrays to zarrs 
This notebook 

- loads the individual zarrs created from each .dat file (using to_individual_zarr.ipynb), 
- computes the stacked profiles and adds them to the dataset
- rechunks to a reasonable chunk size in the time dimension, and
- write the whole thing to a zarr store in the ldeo-glaciology bucket.

Import packages

In [1]:
import numpy as np
from dask.distributed import performance_report
import xarray as xr
import fsspec
import json

with open('../../../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:32905")
client

<Client: 'tcp://127.0.0.1:32905' processes=4 threads=16, memory=58.87 GiB>

In [6]:
def zarrs_to_onezarr(site):
    ds = xr.open_mfdataset(f'gs://ldeo-glaciology/apres/greenland/2022/{site}/individual_zarrs_prechunked_3/dat_*',
                               chunks = {}, 
                               engine = 'zarr', 
                               consolidated = False, 
                               parallel = True)
    ds['attenuator'] = ds.attenuator[500]
    ds['AFGain'] = ds.AFGain[500]


    
    for var in ds:
        del ds[var].encoding['chunks']

    profile_stacked = ds.profile.mean(dim='chirp_num')
    ds_stacked = ds.assign({'profile_stacked':profile_stacked})
    ds_stacked_rechunked = ds_stacked.chunk({'time':20})
    
    encoding = {i: {"dtype": "float64"} for i in ds_stacked_rechunked.data_vars}

    filename = f'gs://ldeo-glaciology/apres/greenland/2022/single_zarrs/{site}' 
    mapper = fsspec.get_mapper(filename, mode='w', token=token) 
    with performance_report(f'ds_stacked_rechunked_{site}.html'):
        ds_stacked_rechunked.to_zarr(mapper, consolidated=True, safe_chunks=False, encoding=encoding)

In [7]:
zarrs_to_onezarr("A101")

In [ ]:
zarrs_to_onezarr("A103")

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/dataset.py:2077: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(  # type: ignore


In [ ]:
zarrs_to_onezarr("A104")

In [ ]:
def reload(site):
    filename = f'gs://ldeo-glaciology/apres/greenland/2022/single_zarrs/{site}'
    ds = xr.open_dataset(filename,
        engine='zarr', 
        consolidated=True, 
        chunks={}) 
    return ds

In [ ]:
A101 = reload("A101")
A102 = reload("A102")
A103 = reload("A103")

In [ ]:
print('it all finished!')